# Cross-subject decoding Motor responses 
### (LINDEX v. LMID and RINDEX v. RMID)

#### Takuya Ito
#### 8/17/2020

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import multiprocessing as mp
import scipy.stats as stats
import os
os.environ['OMP_NUM_THREADS'] = str(1)
import statsmodels.sandbox.stats.multicomp as mc
import seaborn as sns
import h5py
import tools as tools
import nibabel as nib

sns.set_style("whitegrid")
plt.rcParams["font.family"] = "FreeSans"


## 0.1 Load data

In [5]:
# Excluding 084
subjNums = ['013','014','016','017','018','021','023','024','026','027','028','030','031','032','033',
            '034','035','037','038','039','040','041','042','043','045','046','047','048','049','050',
            '053','055','056','057','058','062','063','066','067','068','069','070','072','074','075',
            '076','077','081','085','086','087','088','090','092','093','094','095','097','098','099',
            '101','102','103','104','105','106','108','109','110','111','112','114','115','117','119',
            '120','121','122','123','124','125','126','127','128','129','130','131','132','134','135',
            '136','137','138','139','140','141']



projectdir = '/home/ti61/f_mc1689_1/SRActFlow/'

# Using final partition
networkdef = np.loadtxt(projectdir + 'data/network_partition.txt')
networkorder = np.asarray(sorted(range(len(networkdef)), key=lambda k: networkdef[k]))
networkorder.shape = (len(networkorder),1)
# network mappings for final partition set
networkmappings = {'fpn':7, 'vis1':1, 'vis2':2, 'smn':3, 'aud':8, 'lan':6, 'dan':5, 'con':4, 'dmn':9, 
                   'pmulti':10, 'none1':11, 'none2':12}
networks = networkmappings.keys()

xticks = {}
reorderednetworkaffil = networkdef[networkorder]
for net in networks:
    netNum = networkmappings[net]
    netind = np.where(reorderednetworkaffil==netNum)[0]
    tick = np.max(netind)
    xticks[tick] = net

## General parameters/variables
nParcels = 360
nSubjs = len(subjNums)

glasserfile2 = projectdir + 'data/Q1-Q6_RelatedParcellation210.LR.CorticalAreas_dil_Colors.32k_fs_RL.dlabel.nii'
glasser2 = nib.load(glasserfile2).get_data()
glasser2 = np.squeeze(glasser2)

sortednets = np.sort(list(xticks.keys()))
orderednetworks = []
for net in sortednets: orderednetworks.append(xticks[net])
    
networkpalette = ['royalblue','slateblue','paleturquoise','darkorchid','limegreen',
                  'lightseagreen','yellow','orchid','r','peru','orange','olivedrab']
networkpalette = np.asarray(networkpalette)

OrderedNetworks = ['VIS1','VIS2','SMN','CON','DAN','LAN','FPN','AUD','DMN','PMM','VMM','ORA']

In [6]:
# gsr = True
nResponses = 2
data_task_rh = np.zeros((len(glasser2),nResponses,len(subjNums)))
data_task_lh = np.zeros((len(glasser2),nResponses,len(subjNums)))

scount = 0
for subj in subjNums:
    data_task_rh[:,:,scount] = tools.loadMotorResponses(subj,hand='Right')
    data_task_lh[:,:,scount] = tools.loadMotorResponses(subj,hand='Left')
    scount += 1

In [7]:
roi_ind_rh = []
rois_rh = [8, 9, 52]
for roi in rois_rh:
    roi_ind_rh.extend(np.where(glasser2==roi+1)[0])
    
roi_ind_lh = []
rois_lh = [188, 189, 232]
for roi in rois_lh:
    roi_ind_lh.extend(np.where(glasser2==roi+1)[0])    
    
roi_ind_lh = np.asarray(roi_ind_lh)
roi_ind_rh = np.asarray(roi_ind_rh)

## Significant activations for each finger confined to motor cortex

In [10]:
smnROIs = np.where(networkdef==networkmappings['smn'])[0] 

ind = []

for roi in smnROIs:
    roi_ind = np.where(glasser2==roi+1)[0]
    ind.extend(roi_ind)
#for roi in smnrois:
#    roi_ind = np.where(glasser2==roi+1)[0]
#    ind.extend(roi_ind)

ind = np.asarray(ind)

out_array = np.zeros((len(glasser2),))
t, p = stats.ttest_rel(data_task_rh[roi_ind_rh,0,:],data_task_rh[roi_ind_rh,1,:],axis=1)
h0 = mc.fdrcorrection0(p)
# tmp_ind = np.intersect1d(ind, roi_ind_rh)
out_array[roi_ind_rh] = h0[0]
        
outputfilename = projectdir + 'data/results/MAIN/MotorRegionsMasksPerSubj/sractflow_smn_outputRH_mask'
tools.mapBackToSurface(out_array,outputfilename)

out_array = np.zeros((len(glasser2),))
t, p = stats.ttest_rel(data_task_lh[roi_ind_lh,0,:],data_task_lh[roi_ind_lh,1,:],axis=1)
h0 = mc.fdrcorrection0(p)
# tmp_ind = np.intersect1d(ind, roi_ind_lh)
out_array[roi_ind_lh] = h0[0]
        
outputfilename = projectdir + 'data/results/MAIN/MotorRegionsMasksPerSubj/sractflow_smn_outputLH_mask'
tools.mapBackToSurface(out_array,outputfilename)

# Save out t-values

In [11]:
smnROIs = np.where(networkdef==networkmappings['smn'])[0] 

ind = []

for roi in smnROIs:
    roi_ind = np.where(glasser2==roi+1)[0]
    ind.extend(roi_ind)

ind = np.asarray(ind)

out_array = np.zeros((len(glasser2),))
t, p = stats.ttest_rel(data_task_rh[roi_ind_rh,0,:],data_task_rh[roi_ind_rh,1,:],axis=1)
h0 = mc.fdrcorrection0(p)
# tmp_ind = np.intersect1d(ind, roi_ind_rh)
out_array[roi_ind_rh] = np.multiply(h0[0],t)
        

outputfilename = projectdir + 'data/results/MAIN/MotorRegionsMasksPerSubj/sractflow_smn_outputRH_mask_tvals'
tools.mapBackToSurface(out_array,outputfilename)

out_array = np.zeros((len(glasser2),))
t, p = stats.ttest_rel(data_task_lh[roi_ind_lh,0,:],data_task_lh[roi_ind_lh,1,:],axis=1)
h0 = mc.fdrcorrection0(p)
# tmp_ind = np.intersect1d(ind, roi_ind_lh)
out_array[roi_ind_lh] = np.multiply(h0[0],t)
        
outputfilename = projectdir + 'data/results/MAIN/MotorRegionsMasksPerSubj/sractflow_smn_outputLH_mask_tvals'
tools.mapBackToSurface(out_array,outputfilename)